In [1]:
import gymnasium as gym
import os
import sys
sys.path.append("./jsbgym")
import jsbgym
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.vec_env import SubprocVecEnv

In [2]:
env = gym.make("A320-AltitudeHoldTask-Shaping.STANDARD-NoFG-v0", render_mode="human")

CHECKPOINT_DIR = "./train/engine_alt_hold/"
LOG_DIR = "./logs/engine_alt_hold/"

env.reset()
env.render()



     JSBSim Flight Dynamics Model v1.2.3 Jun  7 2025 19:28:34
            [JSBSim-ML v2.0]

JSBSim startup beginning ...



In [ ]:
gamma = 0.99

env = gym.make("A320-HdotHoldTask-Shaping.STANDARD-NoFG-v0", render_mode=None, gamma = gamma)
# env = gym.make("A320-HdotHoldTask-Shaping.STANDARD-NoFG-v0", render_mode="human", gamma = gamma)
# env = gym.make("A320-HdotHoldTask-Shaping.STANDARD-FG-v0", render_mode="human_fg")

CHECKPOINT_DIR = "./train/engine_hdot_hold/"
LOG_DIR = "./logs/engine_hdot_hold/"

env.reset()
env.render()



     JSBSim Flight Dynamics Model v1.2.3 Jun  7 2025 19:28:34
            [JSBSim-ML v2.0]

JSBSim startup beginning ...



In [ ]:
# Attempt at parallel simulation, didn't work, I think because of JSBSim
def make_env():
    def _init():
        env = gym.make("A320-HdotHoldTask-Shaping.STANDARD-NoFG-v0", render_mode=None, gamma = gamma)
        # env.reset()
        return env
    return _init

n_envs = 4
env = SubprocVecEnv([make_env() for _ in range(n_envs)])

In [ ]:
for episode in range(1, 6):
    obs, info = env.reset()
    done = False
    total_reward = 0
    while not done:
        obs, reward, done, _, info = env.step(env.action_space.sample())
        total_reward += reward
    print("Total Reward for episode {} is {}".format(episode, total_reward))

In [4]:
class TrainAndLoggingCallback(BaseCallback):
    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(
                self.save_path, "best_model_{}".format(self.n_calls)
            )
            self.model.save(model_path)

        return True

In [5]:
callback = TrainAndLoggingCallback(check_freq=100000, save_path=CHECKPOINT_DIR)
model = PPO("MlpPolicy", env, tensorboard_log=LOG_DIR, device="cpu", gamma=gamma)

In [8]:
model.learn(total_timesteps=1000000, callback=callback)
model.save("JSBSim_1000000_steps")

## Visualization

In [ ]:
env = gym.make("A320-AltitudeHoldTask-Shaping.STANDARD-FG-v0", render_mode="human_fg")
env.reset()
model = PPO.load("JSBSim_1000000_steps.zip", device="cpu")

In [18]:
env = gym.make("A320-HdotHoldTask-Shaping.STANDARD-FG-v0", render_mode="human_fg")
env.reset()
# model = PPO.load(os.path.join("train/engine_hdot_hold", "best_model_100000"), device="cpu")
model = PPO.load("JSBSim_1000000_steps.zip", device="cpu")



     JSBSim Flight Dynamics Model v1.2.3 Jun  7 2025 19:28:34
            [JSBSim-ML v2.0]

JSBSim startup beginning ...



/home/pietbcm/diff_thrust_rl/jsbgym/jsbgym/environment.py:131: UserWarning: If training, use NoFG instead of FG in the env_id. Using FG will cause errors while training after a while.
  warnings.warn(


In [19]:
env.render()

In [ ]:
for episode in range(1, 6):
    obs, _ = env.reset()
    done = False
    total_reward = 0
    while not done:
        action, _ = model.predict(obs)
        obs, reward, done, _, info = env.step(action)
        total_reward += reward
    print("Total Reward for episode {} is {}".format(episode, total_reward))

In [1]:
env.close()

NameError: name 'env' is not defined